<h1 align="center">Práctica 9. El modelo "Map Reduce"</h1>
<h3 style="display:block; margin-top:5px;" align="center">Estructuras de datos</h3>
<h3 style="display:block; margin-top:5px;" align="center">Grado en Ciencia de Datos</h3>
<h3 style="display:block; margin-top:5px;" align="center">2023-2024</h3>    
<h3 style="display:block; margin-top:5px;" align="center">Universitat Politècnica de València</h3>
<br>

**Pon/poned aquí tú/vuestros nombre(s):**
- Pablo Pertusa Canales

## Índice
1. ### [Introducción: Objetivos de la práctica](#intro)
1. ### [Actividad 1: Emparejando, *una vez más*, tuercas y tornillos...](#act1)
1. ### [Actividad 2: Sumar la lluvia de cada mes...](#act2)

<a id='intro'></a>
## Introducción: Objetivos de la práctica

El objetivo de esta práctica consiste en familiarizarse con el modelo de programación *Map Reduce*. No pretende mostrar que sea más eficiente a nivel temporal porque no vamos a paralelizar la ejecución.

Tampoco es objetivo de esta práctica aprender a utilizar un *framework* concreto ni medir el coste de las comunicaciones involucradas en el proceso (resulta interesante ver cómo, por ejemplo, para la multiplicación de matrices, hay soluciones con un menor coste en comunicaciones que otras, como se explica en [este libro](http://www.mmds.org/)).

Para ello, vamos a realizar varias actividades. En todas ellas deberás crear una clase que herede de `MapReduce` que mostramos a continuación y que es idéntico al visto en clase de teoría (con lo que dispones de varios ejemplos de uso de la misma):

In [10]:
import collections

class MapReduce:
    def __call__(self, inputdata):
        map_responses = collections.defaultdict(list)
        for (k1,v1) in inputdata:
            for (k2,v2) in self.mapper(k1,v1):
                map_responses[k2].append(v2)
        reduce_responses = collections.defaultdict(list)
        for k2,listv2 in map_responses.items():
            for v2 in self.reducer(k2,listv2):
                reduce_responses[k2].append(v2)
        return reduce_responses

<a id='act1'></a>
## Actividad 1: Emparejando, *una vez más*, tuercas y tornillos...

En este caso no hace falta que tengamos una clase `Tuerca` y otra `Tornillo`. Simplemente vamos a representarlos como tuplas con la cadena `'Tuerca'` o `'Tornillo'` y un radio que será un número entero entre 50 y 70.

Una tuerca se puede emparejar con un tornillo si ambos tienen el mismo radio.

Los datos de entrada son una lista totalmente desordenada de tuercas y tornillos:

In [11]:
import random
import numpy as np
%pprint

#N = 100
#lista = [(random.choice(('Tuerca','Tornillo')), random.randint(50, 70)) for dummy in range(N)]
lista = [('Tuerca', 50), ('Tornillo', 70), ('Tuerca', 59), ('Tuerca', 69), ('Tornillo', 63), ('Tuerca', 62), 
         ('Tuerca', 63), ('Tuerca', 59), ('Tuerca', 56), ('Tornillo', 62), ('Tuerca', 57), ('Tuerca', 60), 
         ('Tuerca', 52), ('Tornillo', 51), ('Tuerca', 69), ('Tuerca', 55), ('Tornillo', 65), ('Tuerca', 62),
         ('Tornillo', 64), ('Tornillo', 51), ('Tornillo', 69), ('Tornillo', 60), ('Tuerca', 57), ('Tuerca', 65),
         ('Tornillo', 70), ('Tuerca', 54), ('Tornillo', 52), ('Tornillo', 51), ('Tornillo', 63), ('Tuerca', 50),
         ('Tuerca', 60), ('Tuerca', 64), ('Tornillo', 55), ('Tuerca', 67), ('Tornillo', 61), ('Tuerca', 51),
         ('Tuerca', 64), ('Tornillo', 69), ('Tornillo', 61), ('Tuerca', 54), ('Tuerca', 57), ('Tuerca', 69),
         ('Tuerca', 56), ('Tuerca', 61), ('Tuerca', 51), ('Tornillo', 62), ('Tornillo', 69), ('Tuerca', 51),
         ('Tornillo', 68), ('Tuerca', 55), ('Tornillo', 61), ('Tuerca', 55), ('Tornillo', 60), ('Tuerca', 66),
         ('Tuerca', 66), ('Tuerca', 61), ('Tuerca', 66), ('Tornillo', 70), ('Tuerca', 59), ('Tuerca', 53), 
         ('Tornillo', 68), ('Tuerca', 56), ('Tuerca', 67), ('Tuerca', 64), ('Tornillo', 69), ('Tuerca', 62),
         ('Tuerca', 64), ('Tuerca', 53), ('Tornillo', 62), ('Tornillo', 67), ('Tuerca', 58), ('Tuerca', 68),
         ('Tuerca', 51), ('Tornillo', 65), ('Tuerca', 50), ('Tuerca', 56), ('Tornillo', 67), ('Tornillo', 66), 
         ('Tornillo', 55), ('Tornillo', 63), ('Tornillo', 63), ('Tornillo', 67), ('Tornillo', 70), 
         ('Tornillo', 50), ('Tornillo', 68), ('Tuerca', 70), ('Tornillo', 51), ('Tornillo', 56), ('Tuerca', 64),
         ('Tuerca', 55), ('Tornillo', 59), ('Tornillo', 69), ('Tuerca', 60), ('Tuerca', 63), ('Tuerca', 54),
         ('Tuerca', 53), ('Tuerca', 61), ('Tuerca', 69), ('Tuerca', 54), ('Tuerca', 66)]

Pretty printing has been turned ON


El objetivo consiste en crear una clase `EmparejarTT` que herede de `MapReduce` de manera que podamos hacer:

```python
emparejador = EmparejarTT()
resultado = emparejador(lista)
...
```
De modo que produzca como resultado un diccionario que asocie a cada radio visto en los datos de entrada una tupla con el número de tuercas y de tornillos que han aparecido, respectivamente. Para el ejemplo de arriba el resultado sería:

```python
defaultdict(<class 'list'>,
            {50: [(3, 1)], 70: [(1, 4)], 59: [(3, 1)],
             69: [(4, 5)], 63: [(2, 4)], 62: [(3, 3)],
             56: [(4, 1)], 57: [(3, 0)], 60: [(3, 2)],
             52: [(1, 1)], 51: [(4, 4)], 55: [(4, 2)],
             65: [(1, 2)], 64: [(5, 1)], 54: [(4, 0)],
             67: [(2, 3)], 61: [(3, 3)], 68: [(1, 3)],
             66: [(4, 1)], 53: [(3, 0)], 58: [(1, 0)]})
```

Tras un post-proceso (**posterior a la llamada** a la parte *map-reduce*), debes imprimir el siguiente resultado:

```
Hay 31 tuercas y tornillos emparejados
Han quedado 10 tornillos sin emparejar
Han quedado 28 tuercas sin emparejar
```

Para saber el nº de tuercas y tornillos emparejados y de tuercas y tornillos sin emparejar has de procesar los de cada radio y luego sumar los de cada tipo de todos los radios.

In [12]:
class EmparejarTT(MapReduce):
    # COMPLETAR
    def mapper(self, t, radio):
        if t == 'Tuerca':
            yield (radio, (1,0))
        else:
            yield (radio, (0,1))
    
    def reducer(self, radio, lista):
        yield tuple(sum(x) for x in zip(*lista))

# COMPLETAR la llamada a un objeto de tipo EmparejarTT y el posproceso
# del diccionario devuelto para poder imprimir lo siguiente:
emparejador = EmparejarTT()
resultado = emparejador(lista)

emparejados = 0
tornsueltos = 0
tuersueltas = 0

for radio, lista in resultado.items():
    nTuer, nTor = lista[0]
    emparejados += min(nTuer, nTor)
    if nTuer > nTor:
        tuersueltas += nTuer - nTor
    elif nTuer < nTor:
        tornsueltos += nTor - nTuer
    
print(f'Hay {emparejados} tuercas y tornillos emparejados')
print(f'Han quedado {tornsueltos} tornillos sin emparejar')
print(f'Han quedado {tuersueltas} tuercas sin emparejar')

Hay 31 tuercas y tornillos emparejados
Han quedado 10 tornillos sin emparejar
Han quedado 28 tuercas sin emparejar


<a id='act2'></a>
## Actividad 2: Sumar la lluvia de cada mes...

En esta actividad debes procesar el resultado de un fichero `'lluvias.txt'` que hemos dejado adjunto en la carpeta de la práctica. Este fichero contiene, en cada línea, el año, el mes, el día y el número de litros de lluvia que cayeron en esa fecha (en un lugar concreto, se entiende). Tiene el siguiente aspecto:

```
2007  1  1 0.00
2007  1  2 0.00
2007  1  3 0.00
2007  1  4 0.00
2007  1  5 153.00
...
2011 12 26 97.00
2011 12 27 0.00
2011 12 28 0.00
2011 12 29 0.00
2011 12 30 0.00
2011 12 31 0.00
```

Y contiene las lluvias de cada día entre los años 2007 y 2011 inclusive.

El objetivo de esta actividad consiste en utilizar el modelo *Map Reduce* para localizar lo que ha llovido en cada uno de los 12 meses **del año elegido como parámetro**, por ejemplo:

```python
with open('lluvias.txt') as f:
    agr = AgregarLluvias(2009)
    resul = agr(enumerate(f)) # enumerate para que ponga una clave a cada línea
    print(resul)
```

El resultado, para 2009, sería:

```
defaultdict(<class 'list'>,
            {1: [1088.0],  2: [ 748.0],  3: [ 884.0],  4: [1034.0],
             5: [ 918.0],  6: [ 764.0],  7: [1700.0],  8: [1366.0],
             9: [ 886.0], 10: [1453.0], 11: [ 885.0], 12: [1475.0]})
```

A continuación debes añadir código para mostrar (en general, no sólo para 2009 del ejemplo):

```
El mes que más ha llovido ha sido el 7 con un total de 1700.0 litros
El mes que menos ha llovido ha sido el 2 con un total de 748.0 litros
```

Prueba a cambiar 2009 por otro año entre 2007 y 2011. Por ejemplo, para 2011:

```python
defaultdict(<class 'list'>,
            {1: [1172.0],  2: [ 436.0],  3: [1560.0],  4: [736.0 ],
             5: [1135.0],  6: [1292.0],  7: [ 281.0],  8: [1244.0],
             9: [ 616.0], 10: [ 922.0], 11: [1221.0], 12: [1018.0]})
```

y

```
El mes que más ha llovido ha sido el 3 con un total de 1560.0 litros
El mes que menos ha llovido ha sido el 7 con un total de 281.0 litros
```

In [39]:
class AgregarLluvias(MapReduce):
    def __init__(self, anyo):
        self.anyo = anyo
    def mapper(self, indice, texto):
        lista = texto.replace('\n','').split()
        if int(lista[0]) == self.anyo:
            yield int(lista[1]), float(lista[3])
    def reducer(self, mes, lista):
        yield sum(lista)
        
    
# COMPLETAR crear objeto de tipo AgregarLLuvias, abrir fichero,
# llamar al objeto y realizar un postproceso del diccionario
# devuelto apra mostrar el més que más y que menos ha llovido.
        
with open('lluvias.txt', 'r') as f:
    agr = AgregarLluvias(2011)
    resul = agr(enumerate(f))
    print(resul)

    maximoN = 0
    maximoMes = 0
    
    for mes, n in resul.items():
        n = n[0]
        if n > maximoN:
            maximoN = n
            maximoMes = mes
    
    minimoMes = 0
    minimoN = maximoN
    
    for mes,n in resul.items():
        n = n[0]
        if n < minimoN:
            minimoMes = mes
            minimoN = n

    print(f'El mes que más ha llovido es {maximoMes} con un total de {maximoN} litros')
    print(f'El mes que menos ha llovido es {minimoMes} con un total de {minimoN} litros')



defaultdict(<class 'list'>, {1: [1172.0], 2: [436.0], 3: [1560.0], 4: [736.0], 5: [1135.0], 6: [1292.0], 7: [281.0], 8: [1244.0], 9: [616.0], 10: [922.0], 11: [1221.0], 12: [1018.0]})
El mes que más ha llovido es 3 con un total de 1560.0 litros
El mes que menos ha llovido es 7 con un total de 281.0 litros
